In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lecture 1: Reading Image Data from Files

In [ ]:
image_shape = (300, 300)
img_path='/content/drive/MyDrive/garbage_classification/'

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(img_path, shuffle=True,
                                                            batch_size=32,
                                                            image_size=image_shape,
                                                            subset='training',
                                                            validation_split=0.3,
                                                            seed=1)

validation_dataset = tf.keras.utils.image_dataset_from_directory(img_path,
                                                                 shuffle=True,
                                                                 batch_size=32,
                                                                 image_size=image_shape,
                                                                 subset='validation',
                                                                 validation_split=0.3,
                                                                 seed=1)

# Shrink the training data to a size we can process during lecture.
train_batches = tf.data.experimental.cardinality(train_dataset)
train_dataset = train_dataset.take(train_batches // 10) # Comment out this line to use the full dataset

# Take every 5th entry as the testing dataset.
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

# Let the hardware determine how many pictures to load at a time.
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 15515 files belonging to 12 classes.
Using 10861 files for training.
Found 15515 files belonging to 12 classes.
Using 4654 files for validation.


In [ ]:
image_final_shape = image_shape + (3,) # Add the three colour channels to the image shape

# Lecture 2: Implement the Convolutional Network

In [ ]:
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip('horizontal'),
                                         tf.keras.layers.RandomRotation(0.2)])
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

prediction_layer = tf.keras.layers.Dense(12, activation='softmax')
inputs = tf.keras.Input(shape=image_final_shape)
x = data_augmentation(inputs)
x = tf.keras.layers.Conv2D(filters=32)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(filters=32)(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(filters=32)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.001

model_optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
model_loss=tf.keras.losses.SparseCategoricalCrossentropy()
model_accuracy_metric=tf.keras.metrics.SparseCategoricalAccuracy(name='sparse_categorical_accuracy')

model.compile(optimizer=model_optimizer, loss=model_loss,
              metrics=[model_accuracy_metric])

model_epochs = 10
model_loss, model_accuracy = model.evaluate(validation_dataset)

model_early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,
                                                       restore_best_weights=True,
                                                       monitor='val_loss')

model_history = model.fit(train_dataset, epochs=model_epochs,
                         validation_data=validation_dataset,
                         callbacks=[model_early_stopping])

fine_tune_epochs = 10

model_accuracy = model_history.history['sparse_categorical_accuracy']
model_val_accuracy = model_history.history['val_sparse_categorical_accuracy']

model_loss = model_history.history['loss']
model_val_loss = model_history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(model_accuracy, label='Training Accuracy')
plt.plot(model_val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 2)
plt.plot(model_loss, label='Training Loss')
plt.plot(model_val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.show()